# Data fetching

In [116]:
import pandas as pd
from io import StringIO
import requests
import numpy as np

isocodes = pd.read_csv('../input/countries-iso-codes/wikipedia-iso-country-codes.csv')
isocodes.columns = isocodes.columns.str.replace(' ', '_').str.lower()
isocodes = isocodes.rename({"english_short_name_lower_case": 'country_name'}, axis=1)

# url = 'https://data.humdata.org/dataset/e1a91ae0-292d-4434-bc75-bf863d4608ba/resource/a636aad6-f40a-4a2c-ab83-7625d7b9d64d/download/acaps_covid19_goverment_measures_dataset.xlsx'
# measures = pd.read_excel(url, sheet_name='Database')
# measures.to_csv('acaps.csv', index=False)

measures = pd.read_csv('acaps.csv')
measures.columns = measures.columns.str.lower()
measures['date_implemented'] = pd.to_datetime(measures['date_implemented'])

measures = measures.merge(isocodes, left_on='iso', right_on='alpha-3_code')
measures

,id,country,iso,admin_level_name,pcode,region,log_type,category,measure,targeted_pop_group,...,source,source_type,link,entry_date,alternative source,country_name,alpha-2_code,alpha-3_code,numeric_code,iso_3166-2
0,1.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,2.0,Afghanistan,AFG,Kabul,NaN,Asia,Introduction / extension of measures,Public health measures,Isolation and quarantine policies,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN,Afghanistan,AF,AFG,4,ISO 3166-2:AF
2,3.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Awareness campaigns,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN,Afghanistan,AF,AFG,4,ISO 3166-2:AF
3,4.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Governance and socio-economic measures,Emergency administrative structures activated ...,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN,Afghanistan,AF,AFG,4,ISO 3166-2:AF
4,5.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Social distancing,Limit public gatherings,No,...,AA,Media,https://www.aa.com.tr/en/asia-pacific/coronavi...,2020-03-14,NaN,Afghanistan,AF,AFG,4,ISO 3166-2:AF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8273,6425.0,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Lockdown,Partial lockdown,No,...,BBC,Media,https://www.bbc.com/news/live/world-africa-476...,2020-04-20,NaN,Zimbabwe,ZW,ZWE,716,ISO 3166-2:ZW
8274,7351.0,Zimbabwe,ZWE,NaN,NaN,Africa,Phase-out measure,Social distancing,Public services closure,No,...,Ministry of Information,Social media,https://twitter.com/MinOfInfoZW,2020-04-24,NaN,Zimbabwe,ZW,ZWE,716,ISO 3166-2:ZW
8275,7352.0,Zimbabwe,ZWE,NaN,NaN,Africa,Phase-out measure,Social distancing,Limit public gatherings,Yes,...,Ministry of Information,Social media,https://twitter.com/MinOfInfoZW,2020-04-24,NaN,Zimbabwe,ZW,ZWE,716,ISO 3166-2:ZW
8276,7353.0,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Governance and socio-economic measures,Economic measures,No,...,Ministry of Information,Social media,https://twitter.com/MinOfInfoZW/status/1252946...,2020-04-24,NaN,Zimbabwe,ZW,ZWE,716,ISO 3166-2:ZW


In [16]:
ecdc = (pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
        .assign(date=lambda f: f['date'].pipe(pd.to_datetime))
       )

In [118]:
# Apple mobility
# url = 'https://covid19-static.cdn-apple.com/covid19-mobility-data/2007HotfixDev47/v2/en-us/applemobilitytrends-2020-05-03.csv'
# response = requests.get(url)

# apple_mobility = (pd.read_csv(StringIO(response.content.decode())))
# apple_mobility.to_csv('apple.csv', index=False)

apple_mobility = (pd.read_csv('apple.csv')
                  .drop('alternative_name', axis=1)
                  .set_index(['geo_type', 'region', 'transportation_type'])
                  .rename_axis("date", axis=1)
                  .stack()
                  .rename('change')
                  .reset_index('date')
                  .assign(date=lambda f: pd.to_datetime(f['date']))
                  .set_index('date', append=True)
                  )

location_code = ecdc.groupby(['location', 'iso_code']).first().iloc[:, 0].reset_index().iloc[:,:2]

apple_mobility = (apple_mobility
                  .reset_index()
                  .replace({'UK': 'United Kingdom', 'Republic of Korea': 'South Korea', 'Macao': 'Macau'})  # Only missing is Macao
                  .merge(location_code, left_on='region', right_on='location', how='left')
                 .assign(change=lambda f: f['change'].div(100).sub(1)))

apple_mobility = apple_mobility.query('iso_code.notna()')
apple_mobility

,geo_type,region,transportation_type,date,change,location,iso_code
0,country/region,Albania,driving,2020-01-13,0.0000,Albania,ALB
1,country/region,Albania,driving,2020-01-14,-0.0470,Albania,ALB
2,country/region,Albania,driving,2020-01-15,0.0143,Albania,ALB
3,country/region,Albania,driving,2020-01-16,-0.0280,Albania,ALB
4,country/region,Albania,driving,2020-01-17,0.0355,Albania,ALB
...,...,...,...,...,...,...,...
106731,sub-region,Puerto Rico,driving,2020-04-29,-0.6185,Puerto Rico,PRI
106732,sub-region,Puerto Rico,driving,2020-04-30,-0.5901,Puerto Rico,PRI
106733,sub-region,Puerto Rico,driving,2020-05-01,-0.5426,Puerto Rico,PRI
106734,sub-region,Puerto Rico,driving,2020-05-02,-0.5557,Puerto Rico,PRI


In [122]:
# oxford.columns[mask].str.extract(r'(..)_.*', expand=False)

oxford = pd.read_csv('https://github.com/OxCGRT/covid-policy-tracker/raw/master/data/OxCGRT_latest.csv')
oxford.columns = oxford.columns.map(str.lower).str.replace(' ', '_')
oxford['date'] = pd.to_datetime(oxford['date'], format='%Y%m%d')

oxford

,countryname,countrycode,date,c1_school_closing,c1_flag,c2_workplace_closing,c2_flag,c3_cancel_public_events,c3_flag,c4_restrictions_on_gatherings,...,h3_contact_tracing,h4_emergency_investment_in_healthcare,h5_investment_in_vaccines,m1_wildcard,confirmedcases,confirmeddeaths,stringencyindex,stringencyindexfordisplay,legacystringencyindex,legacystringencyindexfordisplay
0,Aruba,ABW,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.00,0.00
1,Aruba,ABW,2020-01-02,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.00,0.00
2,Aruba,ABW,2020-01-03,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.00,0.00
3,Aruba,ABW,2020-01-04,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.00,0.00
4,Aruba,ABW,2020-01-05,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18908,Kosovo,RKS,2020-04-30,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,NaN,NaN,NaN,NaN,806.0,22.0,83.6,83.6,82.14,82.14
18909,Kosovo,RKS,2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,806.0,22.0,NaN,83.6,NaN,82.14
18910,Kosovo,RKS,2020-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,823.0,22.0,NaN,83.6,NaN,82.14
18911,Kosovo,RKS,2020-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,851.0,22.0,NaN,83.6,NaN,82.14


In [171]:
ordinal_columns = oxford.columns[oxford.columns.str.contains("^c._.*_.*")]
geographic_columns = oxford.columns[oxford.columns.str.contains('^c._flag')]
strip_measure_name = lambda name: name.split('_')[0]
measures_ix = dict(zip(ordinal_columns.map(strip_measure_name), ordinal_columns.map(lambda s: '_'.join(s.split('_')[1:]))))

oxford_long = (oxford
               .set_index(['countrycode', 'date'])
               [geographic_columns]
               .rename(columns=strip_measure_name)
               .rename_axis('variable', axis=1)
               .stack()
               .to_frame('flag')
               .join(oxford.set_index(['countrycode', 'date'])
                     [ordinal_columns]
                     .rename(columns=strip_measure_name)
                     .replace(0, np.nan)
                     .rename_axis('variable', axis=1)
                     .stack()
                     .rename('measure'),
                     how='outer'
                    )
              )

print(oxford_long.shape)
print(measures_ix)
oxford_long.loc['FRA']

(53783, 2)
{'c1': 'school_closing', 'c2': 'workplace_closing', 'c3': 'cancel_public_events', 'c4': 'restrictions_on_gatherings', 'c5': 'close_public_transport', 'c6': 'stay_at_home_requirements', 'c7': 'restrictions_on_internal_movement', 'c8': 'international_travel_controls'}


flag  measure
date       variable               
2020-01-23 c8         NaN      1.0
2020-01-24 c8         NaN      1.0
2020-01-25 c8         NaN      1.0
2020-01-26 c8         NaN      1.0
2020-01-27 c8         NaN      1.0
...                   ...      ...
2020-05-01 c4         1.0      4.0
           c5         1.0      1.0
           c6         1.0      2.0
           c7         1.0      2.0
           c8         NaN      4.0

[458 rows x 2 columns]